# 5.动态规划算法
**动态规划（dynamic programming）** 的基本思想是将待求解问题分解成若干个子问题，先求解子问题并保存已解决的子问题的答案，在求解目标问题的过程中，需要这些子问题答案时就可以直接利用，避免重复计算。
基于**动态规划**的强化学习算法主要有两种：
- **策略迭代（policy iteration）** ，策略迭代中的**策略评估**使用贝尔曼期望方程来得到一个策略的状态价值函数
- **价值迭代（value iteration）**，价值迭代直接使用贝尔曼最优方程来进行动态规划

不同于**4.蒙特卡洛方法**和之后要介绍的**时序差分算法**，基于动态规划的这两种强化学习算法：
- 要求事先知道环境的状态转移函数和奖励函数，也就是需要知道整个马尔可夫决策过程这样的一个白盒环境
- 但是，现实中的白盒环境很少
- 通常只适用于有限马尔可夫决策过程，即状态空间和动作空间是离散且有限的

本节通过两个环境（满足以上条件）来学习这两种算法：
1. **悬崖漫步（Cliff Walking）环境** 
2. **冰湖（Frozen Lake）环境** 

导入相关库

In [23]:
import copy
import gymnasium as gym
import time

## 5.1 悬崖漫步环境:
### Cliff Walking环境示意图（与代码定义不同）:
![Cliff Walking环境示意图](images/悬崖漫步环境示意图.png)
### 规则：
- 智能体在每一个状态采取以下 4 种动作之一：上、下、左、右
- 触碰到边界墙壁则状态不发生改变，否则就会相应到达下一个状态
- 环境中有一段悬崖，智能体掉入悬崖或到达目标状态都会结束动作并回到起点，也就是说掉入悬崖或者达到目标状态是终止状态
- 智能体每走一步的奖励是 −1，掉入悬崖的奖励是 −100。

### Cliff Walking 环境代码定义：

In [24]:
class CliffWalkingEnv:
    """ 悬崖漫步环境"""
    def __init__(self, ncol=12, nrow=4):
        self.ncol = ncol  # 定义网格世界的列
        self.nrow = nrow  # 定义网格世界的行
        # 转移矩阵P[state][action] = [(p, next_state, reward, done)]包含下一个状态和奖励，p为执行某个动作后，转移到下一个状态的概率（本环境都为1）
        self.P = self.createP()

    def createP(self):
        # 初始化状态转移矩阵P (48*4)
        P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]
        # 4种动作, change[0]:上,change[1]:下, change[2]:左, change[3]:右。坐标系原点(0,0)
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        for i in range(self.nrow):
            for j in range(self.ncol):
                for a in range(4):
                    # 位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
                    if i == self.nrow - 1 and j > 0:
                        P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0, True)]
                        continue
                    # 其他位置
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))
                    next_state = next_y * self.ncol + next_x
                    reward = -1
                    done = False
                    # 下一个位置在悬崖或者终点
                    if next_y == self.nrow - 1 and next_x > 0:
                        done = True
                        if next_x != self.ncol - 1:  # 下一个位置在悬崖
                            reward = -100
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
        return P

In [25]:
env = CliffWalkingEnv()
s = 0  # 当前状态
a = 0  # 执行动作 0
for res in env.P[s][a]:
    p, next_state, reward, done = res
    print(f"转移概率: {p}, 下一个状态: {next_state}, 奖励: {reward}, 是否终止: {done}")

转移概率: 1, 下一个状态: 0, 奖励: -1, 是否终止: False


### 5.1.1 策略迭代算法
策略迭代由两部分组成：**策略评估（policy evaluation）** 和**策略提升（policy improvement）**
**策略评估**：计算评估某个策略下的**价值函数**（动作<==>状态）
**策略提升**：依据 **策略提升定理（policy improvement theorem）** 更新动作策略

#### 策略提升定理
由于贪心地在每一个状态选择动作价值最大的动作$\pi^{\prime}(s)$：
$$\pi^{\prime}(s)=\arg\max_aQ^\pi(s,a)=\arg\max_a\{r(s,a)+\gamma\sum_{s^{\prime}}P(s^{\prime}|s,a)V^\pi(s^{\prime})\}$$
此时动作价值函数与状态价值函数无数值差别，即：
$$Q^\pi(s,\pi^{\prime}(s))\geq V^\pi(s)$$
最终会使：
$$V^{\pi^{\prime}}(s)\geq V^\pi(s)$$
**证明：**
$$\begin{aligned}
V^{\pi}(s) & \leq Q^{\pi}(s,\pi^{\prime}(s)) \\
 & =\mathbb{E}_{\pi^{\prime}}[R_t+\gamma V^{\pi}(S_{t+1})|S_t=s] \\
 & \leq\mathbb{E}_{\pi^{\prime}}[R_t+\gamma Q^\pi(S_{t+1},\pi^{\prime}(S_{t+1}))|S_t=s] \\
 & =\mathbb{E}_{\pi^{\prime}}[R_t+\gamma R_{t+1}+\gamma^2V^\pi(S_{t+2})|S_t=s] \\
 & \leq\mathbb{E}_{\pi^{\prime}}[R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\gamma^3V^\pi(S_{t+3})|S_t=s] \\
 & \leq\mathbb{E}_{\pi^{\prime}}[R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\gamma^3R_{t+3}+\cdots|S_t=s] \\
 & =V^{\pi^{\prime}}(s)
\end{aligned}$$

**总体过程：** 对当前的策略进行策略评估，得到其状态价值函数，然后根据该状态价值函数进行策略提升以得到一个更好的新策略，接着继续评估新策略、提升策略……直至最后收敛到最优策略：
$$\pi^0\overset{\text{策略评估}}{\operatorname{\operatorname{\longrightarrow}}}V^{\pi^0}\overset{\text{ 策略提升}}{\operatorname{\operatorname{\longrightarrow}}}\pi^1\overset{\text{策略评估}}{\operatorname{\operatorname{\longrightarrow}}}V\overset{\pi^1}{\operatorname{\operatorname{\longrightarrow}}}\overset{\text{策略提升}}{\operatorname{\operatorname{\longrightarrow}}}\pi^2\overset{\text{策略评估}}{\operatorname{\operatorname{\longrightarrow}}}\ldots\overset{\text{策略提升}}{\operatorname{\operatorname{\longrightarrow}}}\pi^*$$


**收敛性证明：**
$$\begin{aligned}
V^{\pi}(s) 
 & =\sum_{a\in A}\pi(a|s)\left(r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^{\pi}(s^{\prime})\right) \quad (6) 
\end{aligned}$$
$$(1-\gamma)\sum_{t=0}^\infty\gamma^t=1$$
$$V<R\sum_{t=0}^\infty\gamma^t=R/(1-\gamma)$$
显然在有限马尔可夫决策过程中，V存在一个**上界**
再根据策略提升定理，更新后的策略的价值函数满足**单调性**
最后根据实数列的**单调有界准则**，该数列一定收敛，也即是策略迭代算法一定收敛


In [26]:
class PolicyIteration:
    """ 策略迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 状态价值初始化价值为0
        self.pi = [[0.25, 0.25, 0.25, 0.25]
                   for i in range(self.env.ncol * self.env.nrow)]  # 初始化为均匀随机策略
        self.theta = theta  # 策略评估收敛阈值
        self.gamma = gamma  # 折扣因子

    def policy_evaluation(self):  # 策略评估
        cnt = 1  # 计数器
        while 1:
            max_diff = 0
            new_v = [0] * self.env.ncol * self.env.nrow
            for s in range(self.env.ncol * self.env.nrow):
                qsa_list = []  # 计算状态s下的所有Q值
                for a in range(4):
                    qsa = 0
                    
                    for res in self.env.P[s][a]:
                        p, next_state, r, done = res
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))  # 源:第3节公式（5）------- +此加号用于后面的冰湖环境
                        
                    qsa_list.append(self.pi[s][a] * qsa)  # 初始四个动作均分为0.25概率
                new_v[s] = sum(qsa_list)  # 存储当前状态新价值， 源:第3节公式（6）状态价值函数V和动作价值函数Q之间的关系
                
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))
            self.v = new_v
            if max_diff < self.theta: break  # 如果一次评估中各状态价值更改幅度最大的都小于收敛阈值，则满足收敛条件,退出评估迭代
            cnt += 1
        print("策略评估进行%d轮后完成" % cnt)

    def policy_improvement(self):  # 策略提升（针对每个状态的动作策略）
        for s in range(self.env.nrow * self.env.ncol):
            qsa_list = []
            for a in range(4):
                qsa = 0
                for res in self.env.P[s][a]:  # 本环境中仅1次循环
                    p, next_state, r, done = res
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)
            maxq = max(qsa_list)  # 4个值中选最大值
            cntq = qsa_list.count(maxq)  # 计算有几个动作得到了最大的Q值
            # 让这些动作均分概率
            self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]
        print("策略提升完成")
        return self.pi

    def policy_iteration(self):  # 策略迭代
        while 1:
            self.policy_evaluation()
            old_pi = copy.deepcopy(self.pi)  # 将 动作策略概率列表pi 进行深拷贝,方便接下来进行比较
            new_pi = self.policy_improvement()
            if old_pi == new_pi: break

In [27]:
def print_agent(agent, action_meaning, disaster=[], end=[]):
    """ 
    打印策略在每个状态下的价值以及智能体会采取的动作
    用^o<o表示等概率采取向左和向上两种动作，ooo>表示在当前状态只采取向右动作
    """
    print("状态价值：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 为了输出美观,保持输出6个字符
            print('%6.6s' % ('%.3f' % agent.v[i * agent.env.ncol + j]), end=' ')
        print()

    print("策略：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 一些特殊的状态,例如悬崖漫步中的悬崖
            if (i * agent.env.ncol + j) in disaster:
                print('****', end=' ')
            elif (i * agent.env.ncol + j) in end:  # 目标状态
                print('EEEE', end=' ')
            else:
                a = agent.pi[i * agent.env.ncol + j]
                pi_str = ''
                for k in range(len(action_meaning)):
                    pi_str += action_meaning[k] if a[k] > 0 else 'o'
                print(pi_str, end=' ')
        print()

In [28]:
env = CliffWalkingEnv()
theta = 0.001
gamma = 0.9

agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()

action_meaning = ['^', 'v', '<', '>']
print_agent(agent, action_meaning, list(range(37, 47)), [47])

策略评估进行60轮后完成
策略提升完成
策略评估进行72轮后完成
策略提升完成
策略评估进行44轮后完成
策略提升完成
策略评估进行12轮后完成
策略提升完成
策略评估进行1轮后完成
策略提升完成
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


可以看到，智能体最终沿着悬崖边到达终点，为最优策略.

### 5.1.2 价值迭代算法
**策略迭代**中的**策略评估**需要进行很多轮才能收敛得到某一策略的状态函数，这需要很大的计算量，尤其是在状态和动作空间比较大的情况下
**价值迭代算法**可以被认为是一种策略评估只进行了一轮更新的**策略迭代算法**，此时，因为**每一轮**选择的是**最优动作**
根据3.5，价值迭代算法利用的是**贝尔曼最优方程**：
$$V^*(s)=\max_{a\in\mathcal{A}}Q^*(s,a)\quad (7) $$
$$Q^\pi(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^\pi(s^{\prime})\quad (5)$$
所以迭代更新公式为：
$$V^{k+1}(s)=\max_{a\in\mathcal{A}}\{r(s,a)+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)V^k(s^{\prime})\}$$


由于每一轮直接根据**状态价值函数V**选择动作
该方法不存在显式的策略，只需维护一个V
当V趋于**收敛**后，恢复出最优策略即可.

**收敛性证明：**
$$\begin{aligned}
\|V^{k+1}-V^*\|_\infty & =\max_{s\in\mathcal{S}}\left|\max_{a\in\mathcal{A}}\{r(s,a)+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)V^{k}(s^{\prime})\}-\max_{a^{\prime}\in\mathcal{A}}\{r(s,a^{\prime})+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a^{\prime})V^*(s^{\prime})\}\right| \\
 & \leq\max_{s,a}\left|r(s,a)+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)V^{k}(s^{\prime})-r(s,a)-\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)V^*(s^{\prime})\right| \\
 & =\gamma\max_{s,a}|\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)(V^{k}(s^{\prime})-V^*(s^{\prime}))| \\
 & \leq\gamma\max_{s,a}\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)\max_{s^{\prime}}|V^{k}(s^{\prime})-V^*(s^{\prime})| \\
 & =\gamma\|V^{k}-V^*\|_{\infty}
\end{aligned}$$

- $V^{*}$ 为最优价值函数，可在迭代过程中保持不变
- 由于每个阶段仅选择最优动作，所以$max_{s,a}\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)=1$

所以有：
$$\|V^{k+1}-V^*\|_\infty\leq\gamma\|V^k-V^*\|_\infty\leq\cdots\leq\gamma^{k+1}\|V^0-V^*\|_\infty$$
- $\|V^0-V^*\|_\infty$为某个固定值
- $\lim_{k\to\infty}V^{k}=V^{*}$

In [29]:
class ValueIteration:
    """ 价值迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 初始化价值为0
        self.theta = theta  # 价值收敛阈值
        self.gamma = gamma
        # 价值迭代策略
        self.pi = [None for i in range(self.env.ncol * self.env.nrow)]

    def value_iteration(self):
        cnt = 0
        while 1:
            max_diff = 0
            new_v = [0] * self.env.ncol * self.env.nrow
            for s in range(self.env.ncol * self.env.nrow):
                qsa_list = []  # 计算状态s下的所有Q价值
                for a in range(4):
                    qsa = 0
                    for res in self.env.P[s][a]:  # 本环境中仅1次循环
                        p, next_state, r, done = res
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                    qsa_list.append(qsa)  # 这一行和下一行代码是价值迭代和策略迭代的主要区别
                new_v[s] = max(qsa_list)
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))
            self.v = new_v
            if max_diff < self.theta: break  # 如果一次评估中各状态价值更改幅度最大的都小于收敛阈值，则满足收敛条件,退出评估迭代
            cnt += 1
        print("价值迭代一共进行%d轮" % cnt)
        self.get_policy()

    def get_policy(self):  # 根据价值函数导出一个贪婪策略
        for s in range(self.env.nrow * self.env.ncol):
            qsa_list = []
            for a in range(4):
                qsa = 0
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)
            maxq = max(qsa_list)
            cntq = qsa_list.count(maxq)  # 计算有几个动作得到了最大的Q值
            # 让这些动作均分概率
            self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]


In [30]:
env = CliffWalkingEnv()
theta = 0.001
gamma = 0.9

agent = ValueIteration(env, theta, gamma)
agent.value_iteration()

action_meaning = ['^', 'v', '<', '>']
print_agent(agent, action_meaning, list(range(37, 47)), [47])

价值迭代一共进行14轮
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


可以看到，解决同样的训练任务，价值迭代总共进行了数十轮，而策略迭代中的策略评估总共进行了数百轮，价值迭代中的循环次数远少于策略迭代.

## 5.2 冰湖环境:
冰湖是 **OpenAI Gym 库** 中的一个环境。OpenAI Gym 库中包含了很多有名的环境，例如 Atari 和 MuJoCo，并且支持定制自己的环境
在之后的章节中，会使用到更多来自 OpenAI Gym 库的环境  (自 2021 年以来一直维护 Gym 的团队已将所有未来开发转移到 Gymnasium):
### gymnasium库下载:
![gymnasium库下载](images/gymnasium库下载.png)
### Frozen Lake环境示意图（与代码定义不同）:
![Frozen Lake环境示意图](images/冰湖环境示意图.png)
### 规则：
- 智能体在每一个状态采取以下 4 种动作之一：上、下、左、右
- 由于智能体在冰面行走，因此每次行走都有一定的概率滑行到附近的其它状态
- 到达冰洞或目标状态时行走会提前结束
- 每一步行走的奖励是 0，到达目标的奖励是 1



### 使用 Gymnasium 创建 FrozenLake-v0 环境：

In [31]:
# 创建环境
# 使用 gymnasium 创建环境，FrozenLake-v1 是推荐使用的版本,通过图形界面显示环境，适合人类用户观察
env = gym.make("FrozenLake-v1", render_mode="human")

# 获取原始环境（绕过 TimeLimit 包装器）
env = env.unwrapped
# 渲染环境
env.reset()
# 显示初始环境
env.render()
time.sleep(2)

#--------------------------------------------------------------------------------------------------
# 查找冰洞和目标
holes = set()
ends = set()
for s in env.P:  # 16个状态
    for a in env.P[s]:  # 4个动作/状态
        for s_ in env.P[s][a]:  # s_ 是一个元组，包含 [prob, next_state, reward, done]
            if s_[2] == 1.0:  # 获得奖励为1，代表目标
                ends.add(s_[1])
            if s_[3] == True:  # 该位置是冰洞
                holes.add(s_[1])
holes = holes - ends

# 打印冰洞和目标的索引
print("冰洞的索引:", holes)
print("目标的索引:", ends)

# 查看目标左边一格的状态转移信息
for a in env.P[14]:  # 查看目标左边一格的状态转移信息
    print(env.P[14][a])
#-------------------------------------------------------------------------------------------------

# 关闭环境
env.close()


冰洞的索引: {11, 12, 5, 7}
目标的索引: {15}
[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


### 5.2.1 策略迭代算法

In [32]:
theta = 1e-5
gamma = 0.9

agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()

action_meaning = ['<', 'v', '>', '^']
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

策略评估进行25轮后完成
策略提升完成
策略评估进行58轮后完成
策略提升完成
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


**为何反直觉？**
因为智能体会随机滑向其他状态的冰冻湖面
例如，在目标左边一格的状态，采取向右的动作时，它有可能会滑到目标左上角的位置，从该位置再次到达目标会更加困难，所以此时采取向下的动作是更为保险的.

### 5.2.2 价值迭代算法

In [33]:
theta = 1e-5
gamma = 0.9

agent = ValueIteration(env, theta, gamma)
agent.value_iteration()

action_meaning = ['<', 'v', '>', '^']
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

价值迭代一共进行60轮
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


 **结果一致，互相验证.**